# 収集結果をまとめ、市区町村ごとの人物を選択

In [1]:
import json
import pickle
from collections import defaultdict
from statistics import median

import pandas as pd
from SPARQLWrapper import SPARQLWrapper
import requests
from tqdm.notebook import tqdm

## 市区町村、ページ、PV数をマージ

（年間合計PV数を利用）

In [2]:
with open("../data/pref_munic_coords.json") as fp:
    pref_munic_coords = json.load(fp)

with open("../data/workspace/page_locations_clean.json") as fp:
    page_locations_clean = json.load(fp)

with open("../data/workspace/page_views.pickle", "rb") as fp:
     page_views = pickle.load(fp)
assert len(page_locations_clean) == len(page_views)

In [3]:
location_pages = defaultdict(list)
for page, locations in page_locations_clean.items():
    for (pref, munic) in locations:
        location_pages[pref, munic].append(page)

In [4]:
def get_yearly_views(page):
    return sum(page_views[page].values())

In [5]:
pref_munic_pages = {}
for pref, munic_coords in pref_munic_coords.items():
    pref_munic_pages[pref] = {}
    for munic in munic_coords.keys():
            pages = [
                {"page": p, "views": get_yearly_views(p)} for p in location_pages[pref, munic]
            ]
            pref_munic_pages[pref][munic] = sorted(pages, key=lambda p: p["views"], reverse=True)

In [6]:
with open("../data/pref_munic_pages.json", "w") as fp:
    json.dump(pref_munic_pages, fp, indent=2, ensure_ascii=False)

In [7]:
pref_munic_pages["岩手県"]["奥州市"]

[{'page': '大谷翔平', 'views': 4914419},
 {'page': '久慈暁子', 'views': 543372},
 {'page': '一城みゆ希', 'views': 102891},
 {'page': '高橋直純', 'views': 74633},
 {'page': '石川修司', 'views': 24664},
 {'page': '佐藤嗣麻子', 'views': 20857},
 {'page': '千葉慎也', 'views': 3406},
 {'page': '鈴々木保香', 'views': 3132},
 {'page': 'Moto', 'views': 2915},
 {'page': '津田禎', 'views': 1771},
 {'page': '鈴木未来', 'views': 1125}]

## 市区町村ごとのページ（人物）を選択、概要文を取得

In [8]:
sparql = SPARQLWrapper(endpoint="http://ja.dbpedia.org/sparql", returnFormat="json")

def get_abstract(page):
    sparql.setQuery(f"""
        SELECT DISTINCT * 
        WHERE {{
            <http://ja.dbpedia.org/resource/{page}>
            dbo:abstract
            ?o .
            }}
    """)
    res = sparql.query().convert()
    bindings = res["results"]["bindings"]
    if len(bindings) < 1:
        return ""
    return bindings[0]["o"]["value"]

In [9]:
get_abstract("大谷翔平")

'大谷 翔平（おおたに しょうへい、1994年7月5日 - ）は、岩手県奥州市出身のプロ野球選手。右投左打。MLBのロサンゼルス・エンゼルス所属。 2013年にNPB・北海道日本ハムファイターズに入団以降、投手と打者を両立する「二刀流」の選手として試合に出場し、2014年には11勝、10本塁打でNPB史上初となる「2桁勝利・2桁本塁打」を達成。2016年には、NPB史上初となるベストナイン賞、ベストDH賞のダブル受賞に加え、リーグMVPに選出された。投手としての球速165km/hは日本人最速記録である。2019年にはMLBにおいて日本人初のサイクルヒットを達成した。'

---

In [10]:
people_for_map = []

for pref, munic_pages in tqdm(pref_munic_pages.items()):
    for munic, pages in munic_pages.items():
        if not pages:
            continue

        selected = sorted(pages, key=lambda p: p["views"], reverse=True)[0]

        people_for_map.append({
            "page": selected["page"],
            "pref": pref,
            "munic": munic,
            "coords": pref_munic_coords[pref][munic],
            "views": selected["views"],
            "abstract": get_abstract(selected["page"])
        })

len(people_for_map)

  0%|          | 0/47 [00:00<?, ?it/s]

1560

---

DBPediaには概要文が存在しなかった人物の対処

In [11]:
def get_extract(page):
    r = requests.get(f"https://ja.wikipedia.org/api/rest_v1/page/summary/{page}")
    r.raise_for_status()
    return r.json()["extract"]

In [12]:
get_abstract("岡野陽一")

''

In [13]:
get_extract("岡野陽一")

'岡野 陽一 は、日本のお笑い芸人である。福井県敦賀市出身。プロダクション人力舎所属。R-1ぐらんぷり2019ファイナリスト。'

In [14]:
for p in people_for_map:
    if p["abstract"]:
        continue

    print(p["page"])
    extract = get_extract(p["page"])
    assert extract, p
    p["abstract"] = extract

池髙暢希
高橋昌幸
西村穣
王林_(アイドル)
天野♥こころ
三浦春馬
入江大生
髙橋光成
伊藤将司
HANAKO
SANADA
山﨑颯一郎
岡野陽一
UTAMARO
髙橋宏斗
栗林良吏
髙田萌生
山縣亮太
三好大倫
貞包里穂
髙橋大悟


---

In [15]:
with open("../data/people_for_map.json", "w") as fp:
    json.dump(people_for_map, fp, indent=2, ensure_ascii=False)

## 備考

ひとりが複数の市区町村となる場合もある

In [16]:
[(p["pref"], p["munic"]) for p in people_for_map if p["page"] in ("新庄剛志")]

[('福岡県', '福岡市南区'), ('長崎県', '対馬市')]

In [17]:
get_abstract("新庄剛志")

'新庄 剛志（しんじょう つよし、1972年1月28日 - ）は、長崎県対馬市（旧：下県郡美津島町）生まれ、福岡県福岡市南区育ちの野球選手（外野手、内野手・右投右打）、タレント、実業家、クリエイター（商品プロデューサー、画家、YouTuberなど）、地方競馬全国協会 (NAR) の馬主。元妻はタレントの大河内志保。紺綬褒章受章（2007年）。プロ野球選手として阪神タイガース、北海道日本ハムファイターズに所属した他、MLBのニューヨーク・メッツなどでもプレイした元メジャーリーガーでもある。阪神時代の1993年に記録した21歳8ヶ月での先発4番出場は球団の歴代最年少記録（2019年シーズン終了時点）。2001年に日本人野手初のメジャーリーガーとなり、2002年には日本人選手として初めてワールドシリーズ出場を果たした。2006年限りで現役を引退していたが、2019年11月27日付でNPBより自由契約選手として公示された。 愛称は「プリンス」「ツーさん」など（）。また、印象的なプレースタイルや言動、立ち居振る舞いから「記録より記憶に残る選手」とも称される。日本ハム時代は登録名を「SHINJO」としていた。表記法をイニシャル以外でのアルファベットとするのはNPB史上初であった。 2001年より2009年まで個人事務所・株式会社剛ゴーエンタープライズに所属し、所属期間の途中よりは事務所とアワーソングスクリエイティブが業務提携してのタレント活動等のマネジメントを受けた。2008年に株式会社レハサフを設立して、2013年の会社閉鎖まで経営した（）。タレント活動等のマネジメントを自社から受けていた期間を経て、以降のマネジメント契約先は、2012年はプラファー、2013年よりエージェントオフィスタクト、2018年よりZE MANAGEMENT。 2010年ごろにインドネシア・バリ島へ移住した。'